In [1]:
from bs4 import BeautifulSoup as BS
import urllib.request as req
import urllib.parse as pa
import pandas as pd
from tqdm import tqdm_notebook
import time
import re
import math

In [2]:
# 평점 url 얻기 위한 url_open

def url_point_Open(movie_num):
    url = 'https://movie.naver.com/movie/bi/mi/point.nhn?code='+str(movie_num)
    url_open = req.urlopen(url)
    
    return url_open

In [3]:
def url_Open(sub_url, page_num):
    url = 'https://movie.naver.com' + sub_url +'&page='+ str(page_num)
    url_open = req.urlopen(url)
    
    return url_open

In [4]:
movie_num = 193804
soup = BS(url_point_Open(movie_num), 'html.parser')
ifr_url = soup.find('iframe')['src']
ifr_url

'/movie/bi/mi/pointWriteFormList.nhn?code=193804&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'

In [5]:
soup = BS(url_Open(ifr_url, 11), 'html.parser')

In [6]:
count = soup.find('strong', 'total').find('em')
count

<em>646</em>

In [7]:
count = int(count.string.replace(',', ''))

In [8]:
count

646

In [9]:
loop_count = math.ceil(count/10)
loop_count

65

In [10]:
point = []
review = []
date = []

In [ ]:
page_num = 1

while page_num  < loop_count+1:
    soup = BS(url_Open(ifr_url, page_num), 'html.parser')
    result = soup.find(class_='score_result')
    lis = result.find_all('li')
    print(page_num, end=' ')
    
    for lst in lis:
        count_span = len(lst.find(class_='score_reple').find_all('span'))
        count_icon = len(lst.find_all(class_='ico_viewer'))
        count_spo = len(lst.find_all(class_='text_spo'))
        point.append(int(lst.find(class_='star_score').find('em').get_text()))
#         print(count_span, count_icon, count_spo)
        
        if count_icon ==1 and count_spo == 1:
            r = lst.find_all('span')[4].get_text().strip()
        elif count_icon == 1:
            r = lst.find(class_='score_reple').find('p').get_text().strip().replace('\n', '').replace('\r', '').replace('\t', '').replace('관람객', '')
        elif count_spo == 1:
            r = lst.find_all('span')[3].get_text().strip()
        elif count_span == 3:
            try:
                r = lst.find_all('a')[0]['data-src']
            except:
                r = lst.find(class_='score_reple').find('p').get_text().strip()
        elif count_span == 2:
            r = lst.find(class_='score_reple').find('p').get_text().strip()
            
        review.append(r)
        date.append(lst.find(class_='score_reple').find_all('em')[1].get_text())
            
    page_num += 1


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 

In [ ]:
df = pd.DataFrame(dict(zip(['날짜','리뷰', '평점'], [date, review, point])))
df

In [ ]:
df.to_csv('movie_Escape_From_Pretoria.csv', encoding='utf-8')